In [ ]:
import sys
sys.path.append('/Users/vdk/GeneveWork/Code/calibpipe-main-calibpipe/')
from astropy.io import ascii
import numpy as np
import pandas as pd
import matplotlib
import csv
import scipy
import matplotlib.pyplot as plt
from seaborn import histplot
import random
import copy
from os import path, makedirs
from astropy.coordinates import SkyCoord
from astropy.table import Table
import astropy.units as u
from ctapipe.containers import MuonEfficiencyContainer
from ctapipe.coordinates import CameraFrame, TelescopeFrame
from ctapipe.io import EventSource, EventSeeker
import math
from ctapipe.core import TelescopeComponent
from ctapipe.core.traits import (
    List,
    Int,
    FloatTelescopeParameter,
    TelescopeParameter,
    Unicode,
)
from ctapipe.image.cleaning import tailcuts_clean
from ctapipe.image.muon import MuonRingFitter, MuonIntensityFitter
from traitlets.config import Config
from ctapipe.image import ImageProcessor
from ctapipe.image.muon import MuonProcessor
from ctapipe.visualization import CameraDisplay
from ctapipe.instrument import CameraGeometry
from ctapipe.calib import CameraCalibrator
from traitlets.config.loader import Config, FileConfigLoader, JSONFileConfigLoader
import pathlib
from calibpipe.throughput import MuonCalibrator
from traitlets.config import Config
from astropy.time import Time
from astropy.coordinates import EarthLocation,SkyCoord, AltAz 
from traitlets.config import Config
import time






In [ ]:
energies_all = []
counter = 0
for k in range(1,11):
    filename = f'/Users/vdk/output_lst1_12Septv1/corsika_theta_0.0_az_180.0_run{k}.simtel.gz'
    source = EventSource(filename)
    event_iterator = iter(source)
    keks = k*1000
    for i,j in enumerate(event_iterator):
        counter += 1
        energies_all.append(j.simulation.shower.energy)

In [ ]:
len(energies_all)

In [ ]:
counter

In [ ]:
event_container = np.array([])
for k in range(1,11):
    filename = f'/Users/vdk/output_lst1_12Septv1/corsika_theta_0.0_az_180.0_run{k}.simtel.gz'
    source = EventSource(filename)
    event_iterator = iter(source)
    for i,j in enumerate(event_iterator):
        event_container = np.append(event_container, j)

In [ ]:
len(event_container)

In [ ]:
source.subarray.peek()

In [ ]:
energies_all = []
for i in event_container:
    energies_all.append(i.simulation.shower.energy.to_value())

In [ ]:
histplot(energies_all, log_scale = True, bins = 45)

In [ ]:
###
# Algorithm to process events and try to calibrate them
###
int_cut = 0.3
min_pixel = 80

muon_processor_config = Config({"MuonProcessor": 
                                {"RingQuery": 
                                 {"quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
                                                        ["ring_containment", "parameters.containment > 0.5"],
                                                        ["ring_completeness", "parameters.completeness > 0.5"]]}}})

muon_processor_config_2 = Config({"MuonProcessor":
                                {"ImageParameterQuery" :
                                 {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})
muons = []
for k in range(1,11):
    filename = f'/Users/vdk/output_lst1_12Septv1/corsika_theta_0.0_az_180.0_run{k}.simtel.gz'
    source = EventSource(filename)
    event_iterator = iter(source)
    for i,event in enumerate(event_iterator):           
        image_processor = ImageProcessor(source.subarray)
        muon_processor = MuonProcessor(source.subarray, config = muon_processor_config)
        calib = CameraCalibrator(
            image_extractor_type="GlobalPeakWindowSum",
            subarray = source.subarray)
        calib(event)
        image_processor(event)
        muon_processor(event)
        if not np.isnan(event.muon.tel[1].efficiency.optical_efficiency):
            muons.append(event)
            
        

In [ ]:
opt_eff = []
for muon_event in muons:
    opt_eff.append(muon_event.muon.tel[1].efficiency.optical_efficiency)


In [ ]:
muons[0].muon.tel[1].efficiency.optical_efficiency

In [ ]:
np.mean(opt_eff)

In [ ]:
### NEW CLEANING NEW CLEANING NEW CLEANING
####
###
###

int_cut = 0.2
min_pixel = 200

muon_processor_config = Config({"MuonProcessor": 
                                {"RingQuery": 
                                 {"quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
                                                        ["ring_containment", "parameters.containment > 0.5"],
                                                        ["ring_completeness", "parameters.completeness > 0.5"]]}}})

muon_processor_config_2 = Config({"MuonProcessor":
                                {"ImageParameterQuery" :
                                 {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels < {min_pixel}"],
                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

muons_cleaned = []
tels = 1
for k in range(1,11):
    filename = f'/Users/vdk/output_lst1_12Septv1/corsika_theta_0.0_az_180.0_run{k}.simtel.gz'
    source = EventSource(filename)
    event_iterator = iter(source)
    for i,event in enumerate(event_iterator): 
        image_processor = ImageProcessor(source.subarray)
        muon_processor = MuonProcessor(source.subarray, config = muon_processor_config_2)
        calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum",subarray = source.subarray)
        pixel_mask = []
        for k,l in enumerate(event.r0.tel[tels].waveform[1]):
            if np.argmax(l) in range(16,40): 
                pixel_mask.append(True)
            else:
                pixel_mask.append(False)
        working_wave = event.r1.tel[tels].waveform.copy()
        for k,l in enumerate(working_wave):
            if pixel_mask[k]:
                #working_wave[k] = np.zeros(40)     
                working_wave[k] = np.mean(event.r1.tel[tels].waveform, axis = 0)
        event.r1.tel[tels].waveform = working_wave
    
        calib(event)
        image_processor(event)
        muon_processor(event)
        if not np.isnan(event.muon.tel[tels].efficiency.width):
            muons_cleaned.append(event)
      


In [ ]:
# PROCESSING OF CLEANED IMAGES
true_muons_cleaned = []

for event in muons_cleaned:
    if event.dl1.tel[tels].parameters.morphology.n_pixels > 30 and abs(event.muon.tel[tels].parameters.intensity_ratio - 1) < 0.3 and event.muon.tel[tels].parameters.completeness > 0.4 and event.muon.tel[tels].parameters.mean_squared_error.to_value() < 0.01:
        print(event.index.event_id)
        plt.figure(figsize = (9,6))
        camgeom = source.subarray.tel[tels].camera.geometry
        title=f"CleanedRing{event.index.event_id}_optEff{round(event.muon.tel[tels].efficiency.optical_efficiency,3)}_width{round(event.muon.tel[tels].efficiency.width.to_value(),3)}"
        disp = CameraDisplay(camgeom,title=title)
        disp.image = event.simulation.tel[tels].true_image
        disp.cmap = plt.cm.RdBu_r
        disp.add_colorbar()
        disp.set_limits_percent(95)
        #plt.show()
        true_muons_cleaned.append(event)
        
plt.close()

In [ ]:
cleaned_opt_eff = []
for event in true_muons_cleaned:
    cleaned_opt_eff.append(event.muon.tel[1].efficiency.optical_efficiency)

In [ ]:
# Not cleaned dataset
print(np.mean(opt_eff))
print(np.std(opt_eff))
print(len(opt_eff))

In [ ]:
# Cleaned dataset
print(np.mean(cleaned_opt_eff))
print(np.std(cleaned_opt_eff))
print(len(cleaned_opt_eff))

In [ ]:
cleaned_completeness = []
for i in true_muons_cleaned:
    cleaned_completeness.append(i.muon.tel[1].parameters.completeness)

completeness = []
for i in muons:
    completeness.append(i.muon.tel[1].parameters.completeness)

In [ ]:
plt.hist(completeness,bins = 7, label = 'completeness of muon rings', histtype = 'step')
plt.hist(cleaned_completeness,bins = 7, alpha = 1, label = 'completeness of cleaned muon rings', histtype = 'step')
plt.xlabel("counts")
plt.ylabel("completeness")
plt.legend()

In [ ]:
len(cleaned_completeness)

In [ ]:
from astropy.table import Table

table = {}

all_muon_parameters = ["event_id", "energy", "pointing"]

for tel_id in subarray.tels.keys():
        table[tel_id] = Table(names=all_muon_parameters)
table.add_row([obs_id, timestamp, muon_intensity_param.optical_efficiency])

In [ ]:
import lstchain

In [ ]:
import h5py

In [ ]:
filename = f'/Users/vdk/folder_for_test/dl1_corsika_theta_0.0_az_180.0_run1.h5'
with h5py.File(filename,"r") as f:
    print(f"Keys: {f.keys()}")
    a_group_key = list(f.keys())[1]
    print(type(f[a_group_key]))
    data = list(f[a_group_key])
    test = f[a_group_key]

In [ ]:
fi = h5py.File(filename, 'r')

In [ ]:
list(fi.keys())

In [ ]:
def printname(name):
    print(name)
fi.visit(printname)

In [ ]:
#for group in fi['dl1/event/telescope/image/LST_LSTCam'][1]:
for group in fi['dl1/event/telescope/parameters/LST_LSTCam'][1]:
    print(group)
    #for item in group:
    #    print(item)

In [ ]:
len(fi['dl1/event/telescope/image/LST_LSTCam'])

In [ ]:
from astropy.io import fits
#filename_fits = '~/folder_for_test/data_table.fits'
filename_fits = '~/folder_for_test/test_table.fits'

In [ ]:
hdul = fits.open(filename_fits)

In [ ]:
hdul[1].data

In [ ]:
for i in range(len(hdul[1].data)):
    if not np.isnan(hdul[1].data[i][10]):
        print(hdul[1].data[i])

In [ ]:
hdul[1].data

In [ ]:
filename = f'/Users/vdk/output_lst1_12Septv1/corsika_theta_0.0_az_180.0_run1.simtel.gz'
source = EventSource(filename)
event_iterator = iter(source)
counter = 0
for i,event in enumerate(event_iterator):
    counter+=1

In [ ]:
((7.842/2)**2)*math.pi




In [ ]:
filename_2 = f'/Users/vdk/folder_for_test/test.h5'
fi = h5py.File(filename_2, 'r')
list(fi.keys())

In [ ]:
fi.visit(printname)

In [ ]:
fi['dl1/event/telescope/image/LST_LSTCam']

In [ ]:
filename = '/Users/vdk/Downloads/DL1_datacheck_20220628.h5'
fi = h5py.File(filename, 'r')
keys = list(fi.keys())

def printname(name):
    print(name)

# Show all included tables
fi.visit(printname)

In [ ]:
for item in fi['runsummary/_i_table/mu_intensity_mean']:
    print(item)

In [ ]:
for i in fi['runsummary/_i_table/mu_intensity_mean/sorted']:
    print(i)

In [ ]:
fi['runsummary/_i_table/mu_intensity_mean/sorted'][:]

In [ ]:
#fi['runsummary/_i_table/mu_intensity_mean']['indices']
fi['cosmics/_i_table/time/zbounds']

In [ ]:
run = 1000
crab_sim_file = f'/Users/vdk/crab_sim/corsika_theta_16.087_az_108.090_run{run}.simtel.gz'
source = EventSource(crab_sim_file)
event_iterator = iter(source)
counter = 0
for i in event_iterator:
    counter += 1

In [ ]:
counter*500()

In [ ]:
3/10000

In [ ]:
3*3600

In [ ]:
150 / 163500

In [ ]:
3/10000

In [ ]:
1255/10501

In [ ]:
150000

In [ ]:
150000 * 0.0003

In [ ]:
10000 / 0.0003

In [ ]:
33333333 / 1000000

In [ ]:
for run in range(1001, 1100):
    crab_sim_file = f'/Users/vdk/crab_sim/corsika_theta_16.087_az_108.090_run{run}.simtel.gz'
    source = EventSource(crab_sim_file)
    event_iterator = iter(source)
    for i,event in enumerate(event_iterator): 
        #print(event.trigger.tels_with_trigger)

In [ ]:
event_counter = 0
for run in range(1001, 1100):
    crab_sim_file = f'/Users/vdk/crab_sim/corsika_theta_16.087_az_108.090_run{run}.simtel.gz'
    source = EventSource(crab_sim_file)
    event_iterator = iter(source)
    for i,event in enumerate(event_iterator):
        event_counter += 1

In [ ]:
event_counter

In [ ]:
28/31405

In [ ]:
3/6000

In [ ]:
1100*310

In [ ]:
for i in range(1,10):
    print(i*5)

In [ ]:
counter = [1,2,3]
to_save = ['k = ', 100]

to_save.append(counter)
to_save

In [ ]:
all_events = [1,2]
stereo_events = [1,2]
muon_ring_number = [1,2,3]
k = 1
start = 1
stop = 10
#np_arr = np.array(['all events: ', all_events, 'stereo: ', stereo_events, 'muons: ', muon_ring_number, 'run :', k])
np.savetxt(f'/Users/vdk/muon_rings_for_runs{start}-{stop}_4LST_all_events.txt', all_events)
np.savetxt(f'/Users/vdk/muon_rings_for_runs{start}-{stop}_4LST_stereo.txt', stereo_events)
np.savetxt(f'/Users/vdk/muon_rings_for_runs{start}-{stop}_4LST_muon.txt', muon_ring_number)
np.savetxt(f'/Users/vdk/muon_rings_for_runs{start}-{stop}_4LST_run.txt', [k])

In [ ]:
file = '/Users/vdk/test.txt'
for i in range(20):
    open_file = open(file, 'a')
    open_file.writelines(str(i)+',')
    open_file.close()



In [ ]:
for i in range(40):
    print(i)
    for j in range(1,25):
        #print("j ", j)
        if j%5 == 0:
            break
        print("j ", j)

In [ ]:
arr1 = [1,2,3]
arr2 = [4,5]

arr1+arr2